In [389]:
import pandas as pd
import glob
import unidecode
import re

import nltk
from nltk.corpus import stopwords
import os
import spacy

In [390]:
def fusion_csv(liste_csv):
    dataset = pd.read_csv(liste_csv[0])
    for ds in liste_csv[1:]:
        dataset = pd.concat([dataset, pd.read_csv(ds)], ignore_index=True)
    
    return dataset

In [391]:
liste_csv = glob.glob('data'+os.path.sep+'*.CSV')

dataset = fusion_csv(liste_csv)

dataset

,Objet,Corps,De: (nom),De: (adresse),De: (type),A: (nom),A: (adresse),A: (type),Cc: (nom),Cc: (adresse),Cc: (Type),Cci: (nom),Cci: (adresse),Cci: (type),Catégories,Critère de diffusion,Importance,Informations facturation,Kilométrage
0,Votre course de lundi matin en Uber,\t \t <https://d1a3f4spazzrp4.cloudfront.net/...,Reçu Uber,noreply@uber.com,SMTP,oscarbenoit1@gmail.com,oscarbenoit1@gmail.com,SMTP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Normal(e),NaN,NaN
1,Vérifier mes frais d'annulation,/// Veuillez répondre au-dessus de cette ligne...,Uber Support,contact_b92ffc2b-70bb-4116-ab15-6363af4dc0ce@e...,SMTP,oscarbenoit1@gmail.com,oscarbenoit1@gmail.com,SMTP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Normal(e),NaN,NaN
2,Votre course de lundi matin en Uber,\t \t <https://d1a3f4spazzrp4.cloudfront.net/...,Reçu Uber,noreply@uber.com,SMTP,oscarbenoit1@gmail.com,oscarbenoit1@gmail.com,SMTP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Normal(e),NaN,NaN
3,⚡️ ️Ventes flash : -30% sur les bières blondes !,Découvrez vite la sélection.\r\n ‌ ‌ ‌ ‌ ‌ ‌ ‌...,Saveur Bière By PerfectDraft,saveur-biere@m.perfectdraft.com,SMTP,oscarbenoit1@gmail.com,oscarbenoit1@gmail.com,SMTP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Normal(e),NaN,NaN
4,Alerte email : Appartement 3 pièces Paris 11e,<https://www.pap.fr/emails/images/logo-pap-pa...,PAP.fr,users-alertes@pap.fr,SMTP,oscarbenoit1@gmail.com,oscarbenoit1@gmail.com,SMTP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Normal(e),NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,[etudiants] Isolement et aides spécifiques,"Bonjour à tous,\r\n\r\n\r\nJ'espère que cette ...",etudiants-request@utt.fr,etudiants-request@utt.fr,SMTP,etudiants@utt.fr,etudiants@utt.fr,SMTP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Normal(e),NaN,NaN
126,bonjour!,Comment allez-vous?\r\n,tstancheva,tstancheva@tu-sofia.bg,SMTP,thibault.maria@utt.fr;vianney.bayart@utt.fr;os...,thibault.maria@utt.fr;vianney.bayart@utt.fr;os...,SMTP;SMTP;SMTP;SMTP;SMTP;SMTP;SMTP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Normal(e),NaN,NaN
127,CPE : Discord,CPE <https://moodle.utt.fr/course/view.php?id=...,MAURER Thomas (via moodle.utt.fr),noreply@utt.fr,SMTP,BENOIT OSCAR,oscar.benoit@utt.fr,SMTP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Normal(e),NaN,NaN
128,Offre de stage ouvrier GROUP ATLANTIC,"Bonjour,\r\n\r\nVous trouverez ci-joint une of...",Severine AUBRY,severine.aubry@utt.fr,SMTP,lucie.abi_chaaya@utt.fr;martin.ade@utt.fr;mael...,lucie.abi_chaaya@utt.fr;martin.ade@utt.fr;mael...,SMTP;SMTP;SMTP;SMTP;SMTP;SMTP;SMTP;SMTP;SMTP;S...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Normal(e),NaN,NaN


In [392]:
def text_cleaning(text):
    text = unidecode.unidecode(text)
    text = re.sub(r"""[.,/"':!;\\]""", '', text)
    text = re.sub(r"""[0-9]+""", '', text) #removing numbers
    text = re.sub(r"""-""", ' ', text) #uniquement - pour les mots du style "allez-vous"
    text = re.sub('\s+', ' ', text)
    text = text.split(' ')
    return text

In [394]:
def clean_address(text):
    index = len(text)
    text = text.replace('@', ' ')
    text = text[:index-4] + text[index-4:].replace(".", ' ')
    text = text.split(' ')
    return text

In [395]:
def stop_words_french(text):
    stop_words = stopwords.words('french')
    text = [word for word in text if word not in stopwords.words('french') and len(word)>1]
    return text

def stop_words_english(text):
    stop_words = stopwords.words('english')
    text = [word for word in text if word not in stopwords.words('english') and len(word)>1]
    return text

In [396]:
def lemmatization(nlp, texte):
    i = 0
    # On regarde chaque mot dans le texte
    # Chaque mot a le numéro i
    for mot in texte:
        # on va lemmatizer
        doc = nlp(mot)
        for token in doc:
            texte[i] = token.lemma_.lower()
            
        i += 1
    
            
    return texte

In [397]:
def remove_https(corps):

    text_file = open(r'corps.txt', 'w',  encoding="utf-8")
    text_file.write(corps)
    text_file.close()

        
    final_text_file = open(r'final_corps.txt', 'w',  encoding="utf-8")
    reading_text_file = open(r'corps.txt', 'r',  encoding="utf-8")
    for line in reading_text_file:
        if "https" not in line:
            final_text_file.write(line)
            
    final_text_file.close()
    reading_text_file.close()

    with open('final_corps.txt', 'r', encoding="utf-8") as file:
        return file.read()

In [398]:
def columns_treatment(df):
    df = df.iloc[:,[0,1,2,3]]
    df = df.rename(columns={'Objet':'objet','Corps':'corps','De: (nom)':'nom', 'De: (adresse)':'adresse'})
    return df

In [399]:
def data_cleaning(df):
    
    df = columns_treatment(df)
    
    nlp_fr = spacy.load('fr_core_news_md')
    nlp_en = spacy.load('en_core_web_md')
    
    is_english = False

    for i in df.index:
        
        corps = remove_https(df['corps'][i])

        corps = text_cleaning(corps)
        
        objet = text_cleaning(df['objet'][i])

        #stop words cleaning for object
        objet = stop_words_english(objet)
        objet = stop_words_french(objet)
        
        # Lemmatization
        objet_fr = lemmatization(nlp_fr, objet)
        
        objet_en = lemmatization(nlp_en, objet)
        
        # on vérifie si l'objet a été lemmatizé en anglais ou non
        if objet_en != objet:
            is_english = True
        if is_english:
            objet = objet_en
            corps = stop_words_english(corps)
            corps = lemmatization(nlp_en, corps)
        else:
            objet = objet_fr
            corps = stop_words_french(corps)
            corps = lemmatization(nlp_fr, corps)

        df['objet'][i] = objet
        df['corps'][i] = corps

        df['adresse'][i] = clean_address(df['adresse'][i])

    return df

In [400]:
dataset = data_cleaning(dataset)


In [401]:
dataset

,objet,corps,nom,adresse
0,"[votre, course, lundi, matin, uber]","[17, octobre, 2022, bientot, oscar, voici, fac...",Reçu Uber,"[noreply, uber, com]"
1,"[verifier, frais, dannulation]","[vouloir, repondre, dessus, ce, ligne, le, rep...",Uber Support,"[contact_b92ffc2b-70bb-4116-ab15-6363af4dc0ce,..."
2,"[votre, course, lundi, matin, uber]","[17, octobre, 2022, oscar, merci, davoir, util...",Reçu Uber,"[noreply, uber, com]"
3,"[vente, flash, pourcent, biere, blonde]","[decouvrez, vite, selection, jusqua, demain, s...",Saveur Bière By PerfectDraft,"[saveur-biere, m.perfectdraft, com]"
4,"[alerte, mail, appartement, piece, paris, 11]","[un, annoncer, correspondre, recherche, locati...",PAP.fr,"[users-alertes, pap, fr]"
...,...,...,...,...
125,"[], isolement, aide, specifique]","[bonjour, tout, jespere, ce, situation, totale...",etudiants-request@utt.fr,"[etudiants-request, utt, fr]"
126,[bonjour],"[comment, aller, ?]",tstancheva,"[tstancheva, tu-sofia, bg]"
127,"[cpe, discord]","[cher, etudiant, voici, position, dfp, concert...",MAURER Thomas (via moodle.utt.fr),"[noreply, utt, fr]"
128,"[offre, stage, ouvrier, group, atlantic]","[bonjour, vous, trouver, ci, joindre, offrir, ...",Severine AUBRY,"[severine.aubry, utt, fr]"


In [402]:
dataset.head()

,objet,corps,nom,adresse
0,"[votre, course, lundi, matin, uber]","[17, octobre, 2022, bientot, oscar, voici, fac...",Reçu Uber,"[noreply, uber, com]"
1,"[verifier, frais, dannulation]","[vouloir, repondre, dessus, ce, ligne, le, rep...",Uber Support,"[contact_b92ffc2b-70bb-4116-ab15-6363af4dc0ce,..."
2,"[votre, course, lundi, matin, uber]","[17, octobre, 2022, oscar, merci, davoir, util...",Reçu Uber,"[noreply, uber, com]"
3,"[vente, flash, pourcent, biere, blonde]","[decouvrez, vite, selection, jusqua, demain, s...",Saveur Bière By PerfectDraft,"[saveur-biere, m.perfectdraft, com]"
4,"[alerte, mail, appartement, piece, paris, 11]","[un, annoncer, correspondre, recherche, locati...",PAP.fr,"[users-alertes, pap, fr]"
